<a href="https://colab.research.google.com/gist/jdbcode/79814fb15c98327707617771772df9ab/g4g22_ndvi_time_series_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Earth Engine setup

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bCMr_VHSynzfhV9-23PMWs0eodNFxm_juI-ydyc7L6k&tc=k2eLqkSw_DE4lQh5TqpuHRB0VooCussoHZC2zET-i6g&cc=_S6N2jlvTOvzzId60SVbZIUDbmVBq4nfuADGM6z1xz4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXm5rjHWA__jULd4ONQkC_C9uOGKQieuhhHYyEEMzAQKWuf15MAhT1s

Successfully saved authorization token.


Import a tool to view thumbnail images

In [2]:
from IPython.display import Image

Get an image collection

In [3]:
col = (ee.ImageCollection('MODIS/006/MOD13A2')
    .select('NDVI')
    .filterDate('2021-01-01', '2022-01-01'))

Animate to see what we're working with

In [ ]:
Image(url=col.getVideoThumbURL({
    'dimensions': 300,
    'region': ee.Geometry.BBox(-180, -89, 180, 89)
}))

Hard to tell, data are outside of 8-bit range, get some stats for scaling

In [ ]:
minMax = col.filterDate('2021-07-01', '2021-08-01').first().reduceRegion(**{
    'reducer': ee.Reducer.percentile([1, 99]),
    'scale': 10e3,
    'geometry': ee.Geometry.BBox(-180, -89, 180, 89)
})
print(minMax.getInfo())

Alright! now we can see some patterns

In [ ]:
Image(url=col.getVideoThumbURL({
    'dimensions': 300,
    'region': ee.Geometry.BBox(-180, -89, 180, 89),
    'min': 0,
    'max': 9000
}))

A little hard to interpret grayscale, add a self-evident color palette

In [ ]:
Image(url=col.getVideoThumbURL({
    'dimensions': 300,
    'region': ee.Geometry.BBox(-180, -89, 180, 89),
    'min': 0,
    'max': 9000,
    'palette': ['white', 'green']
}))

Good, that's easier to interpret. Not a big fan of the MODIS projection, let's try with World Equidistant Cylindrical

In [ ]:
Image(url=col.getVideoThumbURL({
    'dimensions': 300,
    'region': ee.Geometry.BBox(-180, -89, 180, 89),
    'min': 0,
    'max': 9000,
    'palette': ['white', 'green'],
    'crs': 'EPSG:4087'
}))

Let's focus on Africa

In [ ]:
Image(url=col.getVideoThumbURL({
    'dimensions': 300,
    'region': ee.Geometry.BBox(-18.7, -36.2, 52.2, 38.1),
    'min': 0,
    'max': 9000,
    'palette': ['white', 'green'],
    'crs': 'EPSG:4087'
}))

Now we're on to something, make bigger

In [ ]:
Image(url=col.getVideoThumbURL({
    'dimensions': 500,
    'region': ee.Geometry.BBox(-18.7, -36.2, 52.2, 38.1),
    'min': 0,
    'max': 9000,
    'palette': ['white', 'green'],
    'crs': 'EPSG:4087'
}))

What a neat intra-annual pattern, let's increase the frame rate

In [ ]:
Image(url=col.getVideoThumbURL({
    'dimensions': 500,
    'region': ee.Geometry.BBox(-18.7, -36.2, 52.2, 38.1),
    'min': 0,
    'max': 9000,
    'palette': ['white', 'green'],
    'crs': 'EPSG:4087',
    'framesPerSecond': 12
}))

We can do a better palette

In [ ]:
Image(url=col.getVideoThumbURL({
    'dimensions': 500,
    'region': ee.Geometry.BBox(-18.7, -36.2, 52.2, 38.1),
    'min': 0,
    'max': 9000,
    'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
    ],
    'crs': 'EPSG:4087',
    'framesPerSecond': 12
}))

Awesome! but there is a lot of noise, probably from clouds/masking, these are 16-day composites. Let's create median inter-annual composites for each 16-day period to clean it up.

First step is to add a property to all images that we can join by - we can use "day of year".

In [4]:
full_col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

def add_doy_prop(img):
  doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year');
  return img.set('doy', doy)

full_col = full_col.map(add_doy_prop)

Perform a "saveAll" join to group all images from the same day of year into a list

In [5]:
distinct_doy = full_col.filterDate('2021-01-01', '2022-01-01')
filter = ee.Filter.equals(**{'leftField': 'doy', 'rightField': 'doy'})
join = ee.Join.saveAll('doy_matches')
join_col = ee.Join.saveAll('doy_matches').apply(distinct_doy, full_col, filter)

Here is what a list of same-day images looks like

In [12]:
img_list = join_col.first().get('doy_matches').getInfo()
for img in img_list:
  print(img)

KeyError: ignored

We need to turn these lists into image collections and compute the per-pixel median

In [ ]:
def median_composite(feature):
    doy_col = ee.ImageCollection.fromImages(feature.get('doy_matches'))
    return doy_col.median()

median_col = ee.ImageCollection(join_col.map(median_composite))

Let's see what the animation looks like now

In [ ]:
Image(url=median_col.getVideoThumbURL({
    'dimensions': 500,
    'region': ee.Geometry.BBox(-18.7, -36.2, 52.2, 38.1),
    'min': 0,
    'max': 9000,
    'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
    ],
    'crs': 'EPSG:4087',
    'framesPerSecond': 12
}))

Smoooooth! Looking good, but I think adding hillshade will add some character

In [ ]:
hillshade = ee.Terrain.hillshade(ee.Image('MERIT/DEM/v1_0_3').multiply(50))

ndvi_vis = {
    'min': 0,
    'max': 9000,
    'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
    ],
    'opacity': 0.7
}

def add_hillshade(img):
    return hillshade.blend(img.visualize(**ndvi_vis))

vis_col = median_col.map(add_hillshade)

Image(url=vis_col.getVideoThumbURL({
    'dimensions': 500,
    'region': ee.Geometry.BBox(-18.7, -36.2, 52.2, 38.1),
    'crs': 'EPSG:4087',
    'framesPerSecond': 12
}))